In [25]:
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/aditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
def termFrequency(sentence):
    wordlist=sentence.split()
    wordfreq = [wordlist.count(p) for p in wordlist]
    freqDict = dict(list(zip(wordlist,wordfreq)))
    aux = [[key,freqDict[key]] for key in freqDict]
    #aux.sort()
    #aux.reverse()
    return freqDict
sortedDict = termFrequency("This is a good sentence")
#for s in sortedDict: print(str(s))
print(sortedDict)

{'This': 1, 'is': 1, 'a': 1, 'good': 1, 'sentence': 1}


In [27]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
content = '''
By MATT SCHWARTZ in Houston and WENDELL JAMIESON in New York Daily News Writers\n\nSaturday, October 14th 1995, 4:22AM\n\nBleeding from a massive chest wound, Tejano star Selena cried, "Help me! Help me! I\'ve been shot!" and then named her killer with her dying breath.\n\nShaken witnesses yesterday told a spellbound Houston courtroom how the blood-covered, mortally wounded 23-year-old Hispanic singing sensation burst into the lobby of the Corpus Christi Days Inn last March 31.\n\nGasping for breath, Selena told motel workers that Yolanda Saldivar the president of her fan club shot her once in the back. She begged, "Close the door or she will shoot me again," the witnesses said.\n\nThe testimony came on the third day of Saldivar\'s trial on charges she murdered Selena with a shot from a .38-caliber revolver when the star tried to fire her for embezzling $30,000 from two boutiques she managed for the singer.\n\nAs a paramedic and motel workers recounted Selena\'s last desperate moments, her mother, father and brother sobbed quietly. Saldivar, as she has throughout the trial, stared blankly.\n\nRuben Deleon, the motel\'s sales director, said he knelt over the dying star and asked who shot her.\n\n"She said \'Yolanda Saldivar in room 158,\' " Deleon said.\n\n"She was yelling, \'Help me! Help me! I\'ve been shot,\' " said Rosalinda Gonzalez, an assistant manager. "I asked who shot her. She said the lady in room 158. She moaned. Her eyes rolled up."\n\nFront desk clerk Shawna Vela said she dialed 911 and took the phone with her as she kneeled over the fallen singer, asking her what happened.\n\n"She said \'Yolanda,\' she said \'In room 158,\' " Vela testified.\n\nThe first paramedic on the scene, Richard Fredrickson, testified that he arrived just two minutes after the call but it was already too late.\n\n"The girl was covered with blood," he remembered. "Blood was thick from her neck to her knees, all the way around both sides."\n\nFredrickson couldn\'t even see the mortal wound until he cut off Selena\'s sweatshirt. He felt for a pulse in her neck but could feel only twitching muscles, he said.\n\nMinutes later, as he rode in an ambulance with the now unconscious Selena, he unclenched the dying woman\'s fist and made an ironic discovery.\n\n"When I opened it, a ring fell out," he said. "It was covered with blood."\n\nThe 14-karat gold and diamond ring, topped with a white-gold egg, was a gift from the Grammy winner\'s boutique employes and Saldivar. Police have said Saldivar demanded the ring back. But before Selena could hand it over, she was shot.\n\nThe singer, whose full name was Selena Quintanilla Perez, was born around Easter and collected decorative eggs.\n\nThe defense claims Saldivar, 35, was hysterical and shot Selena by accident. Prosecutors say it was deliberate.
'''
doc = nlp(content)
temp = doc.sents
sentences = []
for sent in temp:
    sentences.append(sent.text)
print(len(sentences))

38


In [28]:
def getSentenceCount(document, word):
    ct=0
    for sent in document:
        wordList = sent.split()
        if wordList.count(word)>0:
            ct+=1
    return ct
#count = getSentenceCount(sentences, "At")
#print(count)

In [30]:
def inverseTermFrequency(document, sentence):
    d=len(document)
    freqDict = termFrequency(sentence)
    sentCount = [getSentenceCount(document,key) for key in freqDict]
    keyList = freqDict.keys()
    idfDict = dict(list(zip(keyList,sentCount)))
    #freqDict,aux = termFrequency(sentence)
    #sentCount = [getSentenceCount(document, key[0]) for key in freqDict]
    #keyList = [key[0] for key in freqDict]
    #idfDict = dict(list(zip(keyList, sentCount)))
    #aux = [[key,idfDict[key]] for key in idfDict]
    #aux.sort()
    #aux.reverse()
    return idfDict
#sortedDict = inverseTermFrequency(sentences, "At least 54 people were arrested on Monday for allegedly trying to clear the barricades")
#for key in sortedDict: print(sortedDict[key])
#print(sortedDict)

In [31]:
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
def sentence_similarity(sent1,sent2, stopwords=None):
    sent=sent1.split() + sent2.split()
    termFrequency1 = termFrequency(sent1)
    termFrequency2 = termFrequency(sent2)
    idf = inverseTermFrequency(sentences, sent1+sent2)
    #print(sent)
    sum1 = 0
    for word in sent:
        a = 0
        if word in termFrequency1.keys():
            a = termFrequency1[word]
        b = 0
        if word in termFrequency2.keys():
            b = termFrequency2[word]
        c = len(sentences)/idf[word]
        sum1+= a*b*c*c
    sum2 = 0
    for word in sent1.split():
        a = termFrequency1[word]
        b = len(sentences)/idf[word]
        sum2+= a*b*b
    sum3 = 0
    for word in sent2.split():
        a = termFrequency2[word]
        b = len(sentences)/idf[word]
        sum3+= a*b*b
    return sum1/(np.sqrt(sum2)*np.sqrt(sum3))
    #return 0
    
#simi = sentence_similarity("Bernstein, 41, and her husband, Doug Flaumenbaum, 33, both recovering crack and heroin addicts, were among three dozen men and women who regained custody of their children." , "They are the third such group to graduate since the program's inauguration in 1998.\n\n")
#print(simi)

In [32]:
stop_words=stopwords.words('english')
def build_similarity_matrix(sentences, stopwords=None):
    S= np.zeros((len(sentences), len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], " "+sentences[idx2], stop_words)
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S
S=build_similarity_matrix(sentences, stopwords)
print(S)

[[0.         0.         0.         ... 0.01318831 0.01701529 0.        ]
 [0.         0.         0.22331074 ... 0.01702501 0.02196532 0.        ]
 [0.         0.02911354 0.         ... 0.         0.         0.        ]
 ...
 [0.00979549 0.02025327 0.         ... 0.         0.11141357 0.04339498]
 [0.01077671 0.02228205 0.         ... 0.09500533 0.         0.01591396]
 [0.         0.         0.         ... 0.06002853 0.02581587 0.        ]]


/home/aditya/Environments/python_ml/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [33]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P
 
#results = pagerank(S)

In [34]:
from operator import itemgetter
#sentence_ranks = pagerank(S)

In [35]:
def textrank(sentences, top_n=5,stopwords=None):
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary
for idx, sentence in enumerate(textrank(sentences, stopwords=stopwords.words('english'))):
    print("%s. %s" % ((idx + 1), ' '.join(sentence)))

/home/aditya/Environments/python_ml/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: 